In [1]:
import sys
sys.path.append('../Storage')

import pgConn
import PostgresSQL_table_queries
import CloudStorage

import pandas as pd
from datetime import datetime

In [2]:
pg_conn = pgConn.PgConn("financial_news")
df = pg_conn.get_financial_news()

Connection to the database successful!
Table name set to: financial_news


In [3]:
df.head()

,id,source,category,headline,href,summary,content,datetime
0,3432723637265220684805700121422338335,Benzinga,Business,'Bitcoin Jesus' Roger Ver Charged With Tax Fra...,https://finance.yahoo.com/news/bitcoin-jesus-r...,Traders often forget about the capital gains t...,'Bitcoin Jesus' Roger Ver Charged With Tax Fra...,2024-05-02T20:45:00.000Z
1,328936130039506379935908987773324306494,Reuters,Business,'Bitcoin Jesus' arrested in Spain on US tax ev...,https://finance.yahoo.com/news/bitcoin-jesus-a...,"An early investor in bitcoins dubbed the ""Bitc...",By Nate Raymond\n(Reuters) - An early investor...,2024-04-30T22:46:23.000Z
2,237036842328241535656762434312155358351,Benzinga,Business,"'Forget Shark Tank, Forget Bitcoin' Kevin O'Le...",https://finance.yahoo.com/news/forget-shark-ta...,Are you looking to build a solid investment po...,In This Article:\nVZ\n+1.93%\n'Forget Shark Ta...,2024-04-29T14:01:31.000Z
3,230512847511832524135336815932283770079,Motley Fool,Business,1 Silly Trick That'll Help You Avoid 1 Serious...,https://finance.yahoo.com/news/1-silly-trick-t...,It's key to maintain a long-term perspective.,Bitcoin (CRYPTO: BTC) is gaining widespread ac...,2024-05-12T12:10:00.000Z
4,330015985554757289964876233928246679182,Motley Fool,Business,"1 Top Cryptocurrency Will Soar 150% By 2025, A...",https://finance.yahoo.com/news/1-top-cryptocur...,As long as money continues to flow into the ne...,"Back in October, investment firm Bernstein pre...",2024-05-04T11:45:00.000Z


In [4]:
def delete_records_for_current_date(df, pg_conn):
    if df is None:
        print("DataFrame is empty. No records to delete.")
        return
        
    # Filter DataFrame for records with today's date
    current_date = datetime.now().strftime("%Y-%m-%d")
    today_records = df[df['datetime'].str.startswith(current_date)]

    # Extract date strings
    date_strings = today_records['datetime'].tolist()

    # Call the delete_records_by_date method
    pg_conn.delete_records_by_date(date_strings)

# Then call the delete_records_for_current_date method
#delete_records_for_current_date(df, pg_conn)
#list_ids = ["11111"]
#pg_conn.delete_records_by_ids(list_ids)

In [5]:
class DataETL():
    
    def __init__(self, dataframe):
            self.df = dataframe
            self.cloudProvider = CloudStorage.CloudStorageProvider()
            
    class Export():
        def __init__(self, dataframe):
            self.df = dataframe
            self.cloudProvider = CloudStorage.CloudStorageProvider()
            
        def export_text_to_s3(self, bucket_name, prefix_path):
            # Initialize AWS storage
            aws_storage = self.cloudProvider.AWS()

            # Create a new bucket
            aws_storage.create_bucket(bucket_name)

            # Upload DataFrame with datetime subfolder structure
            aws_storage.upload_dataframe_with_datetime_subfolders(self.df, bucket_name, prefix_path)
        
        def export_text_to_s3_full_file(self, bucket_name, prefix_path, filename):
            aws_storage = self.cloudProvider.AWS()
            aws_storage.upload_dataframe_to_csv(self.df, bucket_name, filename, prefix_path)
            
    class Ingestion():
        def __init__(self, dataframe):
            self.df = dataframe
            self.cloudProvider = CloudStorage.CloudStorageProvider()
            
        def get_full_data_csv_file(self, bucket_name, prefix_path):
            # Initialize AWS storage
            aws_storage = self.cloudProvider.AWS()
            return aws_storage.get_csv_from_specific_folder(bucket_name, prefix_path)
        
        def get_data_csv_file_by_datetime(self, bucket_name, prefix_path, year, month, day, hour, minute):
            # Initialize AWS storage
            aws_storage = self.cloudProvider.AWS()
            return aws_storage.get_dataframe_from_specific_datetime(bucket_name, prefix_path, year=None, month=None, day=None, hour=None, minute=None)
    
    class Process():
        
        def __init__(self, dataframe):
            self.df = dataframe
        
        def getData():
            self.df = pg_conn.get_financial_news()
        
        def filter_by_current_date(self):
            # Get the current date as a string
            today_date_str = datetime.today().strftime('%Y-%m-%d')

            # Extract the year, month, and day from the current date string
            current_year, current_month, current_day = today_date_str.split('-')

            # Filter the DataFrame by comparing the substrings of the datetime column
            filtered_df = self.df[self.df['datetime'].str.startswith(f'{current_year}-{current_month}-{current_day}')]

            return filtered_df
            
    class Transform():
        def extractStopWords():
            pass

In [6]:
etl = DataETL(df)
etl_process = etl.Process(etl.df)
filtered_df = etl_process.filter_by_current_date()
filtered_df.head()

,id,source,category,headline,href,summary,content,datetime
81,184615634271720255200774755329567199867,Reuters,Business,Dollar steady; ether fuels crypto rally,https://finance.yahoo.com/news/dollar-steady-e...,The dollar was firm on Tuesday while the yen s...,By Rae Wee\nSINGAPORE (Reuters) - The dollar w...,2024-05-21T00:52:56.000Z
82,189762237089382518389536399032437934857,Benzinga,Politics,Donald Trump Is Working on a 'Day 1' Crypto Po...,https://finance.yahoo.com/news/donald-trump-wo...,"Over the past several years, former President ...","Over the past several years, former President ...",2024-05-21T16:00:13.000Z
93,128900555397233729633900027165633604386,Fortune,Business,Ethereum rises 18% and Sen. Elizabeth Warren s...,https://finance.yahoo.com/news/ethereum-rises-...,Crypto boosters are in a fine mood after a ser...,Crypto markets got a jolt on Monday on news th...,2024-05-21T13:47:14.000Z
98,138888913173957286402298880907217363652,CoinDesk,Business,First Mover Americas: Bitcoin Hits $71K as Eth...,https://finance.yahoo.com/news/first-mover-ame...,The latest price moves in bitcoin (BTC) and cr...,In This Article:\nBTC-USD\n+2.24%\nETH-USD\n+2...,2024-05-21T12:03:44.000Z
115,260948872914900172955139133162812307927,CoinDesk,Business,How Bitcoin ETFs Are Changing the Risk-Reward ...,https://finance.yahoo.com/news/bitcoin-etfs-ch...,By approving bitcoin as an underlying product ...,"As anticipated, the rollout of spot bitcoin ex...",2024-05-21T13:45:54.000Z


In [7]:
export_rows_to_s3 = True
etl_export = etl.Export(filtered_df)
bucket_name = "test-financial-news-bucket"
prefix_path = "news/crypto"
if export_rows_to_s3 == True:
    etl_export.export_text_to_s3(bucket_name, prefix_path)

Bucket 'test-financial-news-bucket' created successfully.
Data for row 81 with id '184615634271720255200774755329567199867' uploaded to S3 bucket 'test-financial-news-bucket' under folder 'news/crypto/2024/05/21/00/52/56/184615634271720255200774755329567199867.csv'
Data for row 82 with id '189762237089382518389536399032437934857' uploaded to S3 bucket 'test-financial-news-bucket' under folder 'news/crypto/2024/05/21/16/00/13/189762237089382518389536399032437934857.csv'
Data for row 93 with id '128900555397233729633900027165633604386' uploaded to S3 bucket 'test-financial-news-bucket' under folder 'news/crypto/2024/05/21/13/47/14/128900555397233729633900027165633604386.csv'
Data for row 98 with id '138888913173957286402298880907217363652' uploaded to S3 bucket 'test-financial-news-bucket' under folder 'news/crypto/2024/05/21/12/03/44/138888913173957286402298880907217363652.csv'
Data for row 115 with id '260948872914900172955139133162812307927' uploaded to S3 bucket 'test-financial-news-

In [8]:
post_full_csv = False
if (post_full_csv == True):
    now = datetime.now()
    filename = f"{now.year}-{now.month:02}-{now.day:02}_full_record"
    etl_export.export_text_to_s3_full_file(bucket_name, prefix_path, filename)

In [9]:
ingest_data = False
get_full_file = True
get_by_datetime = False
df_from_file = None

if ingest_data == True:
    etl_ingestion = etl.Ingestion(etl.df)
    bucket_name = "test-financial-news-bucket"
    prefix_path = "news/crypto/"
    year = ''
    mont = ''
    day = ''
    hour = ''
    minute = ''
    if get_full_file == True:
        df_from_file = etl_ingestion.get_full_data_csv_file(bucket_name, prefix_path)
    elif get_by_datetime == True:
        etl_ingestion.get_data_csv_file_by_datetime(bucket_name, prefix_path, year, month, day, hour, minute)

In [10]:
if df_from_file is not None:
    df_from_file.head()